In [1]:
%matplotlib inline 

from collections import defaultdict
import datetime
import glob 
import math
import itertools
import os 

import numpy as np
import pysam
import scipy
import seaborn as sns
from matplotlib import gridspec
import matplotlib
import pandas as pd
from IPython.core.display import HTML
import sklearn

from gscripts import qtools
from gscripts.general import dataviz
from gscripts.encode import encode_helpers

sns.set_style("darkgrid")

img_dir = "/home/gpratt/Dropbox/Pratt_Gabriel/PapersInProgress/eCLIP_qc/working_figures/fig_2"

# Make proper bam file again
(make bam files great again)

In [2]:
print "foo"

foo


In [3]:
fixed_bam_out_dir = "/home/gpratt/projects/idr/analysis/fixed_bam_out_dir"
def fixed_bam_name(fn):
    out_fn = os.path.basename(fn)
    out_fn = ".".join(out_fn.split(".")[:4] + ['rmRep', 'bam'])
    out_file = os.path.join(fixed_bam_out_dir, out_fn)
    return out_file

def fix_eric_sam(fn):
    out_file = fixed_bam_name(fn)
    return "python ~/gscripts/gscripts/clipseq/fix_sam_file.py --sam {} | samtools view -bS - > {}".format(fn, out_file)

def format_rmrep(fn, shuffle=False):
    base_fn = os.path.basename(os.path.splitext(fn)[0])
    base_fn = os.path.join(downsample_path, base_fn)
    if shuffle:
        shuffle_fn = fn + ".shuffled"
        shuffle_code = "samtools bamshuf {0} {1}".format(fn, shuffle_fn)
        #change input to the shuffled, ordered file
        fn = shuffle_fn + ".bam"
    rm_rep_fn = base_fn + ".rmRep.bam"
    metrics_fn = base_fn + ".rmRep.metrics"
    out_fn = base_fn + ".rmRep.sorted.bam"
    collapse_code = "barcode_collapse_pe.py -b {input_fn} -o {rm_rep} -m {metrics} && samtools sort {rm_rep} -o {out_fn} && samtools index {out_fn}".format(input_fn=fn, rm_rep=rm_rep_fn, metrics=metrics_fn, out_fn=out_fn)
    if shuffle:
        return " && ".join([shuffle_code, collapse_code])
    else:
        return collapse_code
    
def format_rmrep_out_name(fn):
    base_fn = os.path.splitext(fn)[0]
    return base_fn + ".rmRep.sorted.bam"

def format_sort(fn):
    base_fn = os.path.splitext(fn)[0]
    out_fn = base_fn + ".sorted.bam"
    return "samtools sort {input_fn} -o {out_fn} && samtools index {out_fn}".format(input_fn=fn, out_fn=out_fn)

def sorted_file_name(fn):
    base_file = os.path.basename(fn)
    stripped_base = os.path.splitext(base_file)[0]
    
    out_file = os.path.join(downsample_path, stripped_base + ".sorted.bam")
    return out_file

def sort_and_index(fn):
    out_file = sorted_file_name(fn)
    intermedeate_file = out_file

    return "samtools sort {0} -o {1} && samtools index {2}".format(fn, intermedeate_file, out_file)

In [4]:
strange_sams = glob.glob("/home/elvannostrand/scratch/ENCODE_20170429_newannotations_FINAL/*.mapped_vs_MASTER_filelist.wrepbaseandtRNA.fa.fixed.fa.UpdatedSimpleRepeat.sam.combined_w_uniquemap.prermDup.sam.gz")
strange_sams += glob.glob("/home/elvannostrand/scratch/ENCODE_20170429_newannotations_FINAL/*.combined_w_uniquemap.rmDup.sam.gz")

In [5]:
sams_to_fix = [fix_eric_sam(fn) for fn in strange_sams if not os.path.exists(fixed_bam_name(fn))]

In [6]:
num_slices = 500
full_process_list = []
for x in range(num_slices):
    cur_list = sams_to_fix[x::num_slices]
    if len(cur_list) > 0:
        full_process_list.append(" && ".join(cur_list))

In [7]:
#571_merged are files we generated, but don't actually use.  Ignoring for now, just using regular 571 files

In [8]:
job_name = "make_proper_bams"
job = qtools.Submitter(commands=full_process_list, 
                 job_name="{}".format(job_name), 
                 sh_filename="/home/gpratt/projects/idr/scripts/{}.sh".format(job_name),
                array=True,
                walltime="6:00:00",
                out_filename="/home/gpratt/projects/idr/scripts/{}.out".format(job_name),
                err_filename="/home/gpratt/projects/idr/scripts/{}.err".format(job_name),
                queue="home-yeo")
job.job()

print "/home/gpratt/projects/idr/scripts/{}.sh".format(job_name)

/home/gpratt/projects/idr/scripts/make_proper_bams.sh


running 0 tasks as an array-job.


In [9]:
collapsed_bams = glob.glob("/home/elvannostrand/scratch/ENCODE_20170429_newannotations_FINAL/*.combined_w_uniquemap.rmDup.sam.gz")
sams_to_fix = [format_sort(fixed_bam_name(fn)) for fn in collapsed_bams if os.path.exists(fixed_bam_name(fn))]

In [10]:
job_name = "sort_family_map_rmdup"
job = qtools.Submitter(commands=sams_to_fix, 
                 job_name="{}".format(job_name), 
                 sh_filename="/home/gpratt/projects/idr/scripts/{}.sh".format(job_name),
                array=True,
                walltime="6:00:00",
                out_filename="/home/gpratt/projects/idr/scripts/{}.out".format(job_name),
                err_filename="/home/gpratt/projects/idr/scripts/{}.err".format(job_name),
                queue="home-yeo")
job.job()

print "/home/gpratt/projects/idr/scripts/{}.sh".format(job_name)

running 500 tasks as an array-job.
job ID: 8864890
running 500 tasks as an array-job.
job ID: 8864891
running 143 tasks as an array-job.


/home/gpratt/projects/idr/scripts/sort_family_map_rmdup.sh


job ID: 8864892


# Scale to all encode

In [11]:
#original path
#downsample_path = "/home/gpratt/projects/idr/analysis/downsample_unshuffled/"
downsample_path = "/home/gpratt/projects/idr/analysis/downsample_unshuffled_v2/"

In [12]:
merged_data = encode_helpers.get_merged_data()
#Filter out anything sequenced after 16/7/18
merged_data[merged_data['Submitted Date'] < datetime.date(2016, 7, 18)]
#Filter out anything not_qced
merged_data = merged_data[merged_data.is_qced]

In [13]:
merged_data['file_id'] = merged_data.CLIP.apply(lambda x: "_".join(os.path.basename(x).split("_")[:-1]))
merged_data_reindex = merged_data.reset_index()
merged_data_reindex['short_rep'] = merged_data_reindex.rep.apply(lambda x: "01" if x == "rep1" else "02")

In [14]:
ip_file_list_foo = pd.read_csv("/home/gpratt/Dropbox/encode_integration/for_eric/full_IP_file_list.csv", index_col=0)


In [15]:
ip_file_list = pd.read_csv("/home/gpratt/Dropbox/encode_integration/for_eric/full_IP_file_list.csv", index_col=0)

#File names that don't match with master file, fixing here
ip_file_list = ip_file_list.replace("TRNC6A", "TNRC6A")
ip_file_list = ip_file_list.replace("BOP1", "NOLC1")
ip_file_list = ip_file_list.replace("EIF2B", "EIF2S2")
ip_file_list = ip_file_list.replace("RO60-TROVE2", "TROVE2")
ip_file_list = ip_file_list.replace("KHDRBS1-SAM68", "KHDRBS1")
ip_file_list = ip_file_list.replace("ADAR1", "ADAR")
ip_file_list = ip_file_list.replace("RDBP", "TARDBP")
ip_file_list = ip_file_list.replace("PKM2", "PKM")

#Only consider data that is in the final dataset
bam_ids = merged_data.file_id.values

ip_file_list = ip_file_list[['ENCODE_ID', 'RBP', 'processing_name_1', 'processing_name_2']]
ip_file_list = ip_file_list[ip_file_list.ENCODE_ID.isin(bam_ids)]

#The rule is always match the final datasets format, or a tidy version of that format, sometimes file names don't match 
#actual uID 203_01 is actually 271_01, we fix that here
#Need to convert ENCODE_ID to uID + rep number
result = []
for name, row in ip_file_list.iterrows():
    new_id = merged_data_reindex[merged_data_reindex.file_id == row.ENCODE_ID].iloc[0]
    result.append("_".join([new_id.uID, new_id.short_rep]))
    
ip_file_list['ENCODE_ID'] = result

ip_file_list = ip_file_list.set_index(["ENCODE_ID", "RBP"])
ip_file_list.columns = pd.MultiIndex.from_tuples((("processing_name", 1.0, "rep1"), ("processing_name", 1.0, "rep2")), 
                                                 names=['CLIP', "fraction", "rep"])

ip_file_list = ip_file_list.stack().stack()

In [16]:
print "these should all be equal, problem if otherwise"
print len(merged_data), len(bam_ids), len(ip_file_list) / 2.

these should all be equal, problem if otherwise
654 654 654.0


In [17]:
def make_single_base(fn):
    r1_name = fn.split(";")[0]
    base_fn = os.path.basename(r1_name)
    return ".".join(base_fn.split(".")[:2])

#original path
#master_path = "/projects/ps-yeolab3/encode/analysis/encode_master/"
master_path = fixed_bam_out_dir

def make_unshuffled_bam(basename):
    return os.path.join(master_path, basename + ".adapterTrim.round2.rmRep.bam")

def make_rmdup_bam(basename):
    #This is a typo I'll want to fix in the full run
    return os.path.join(downsample_path, basename + ".adapterTrim.round2.rmRep.rmRep.sorted.bam")

In [18]:
ip_file_list['basename'] = ip_file_list.processing_name.apply(make_single_base)
ip_file_list['unshuffled_bam'] = ip_file_list.basename.apply(make_unshuffled_bam)
ip_file_list['rmdup_bam'] = ip_file_list.basename.apply(make_rmdup_bam)


# Make all the Downsampled bams

In [19]:
ip_file_list = ip_file_list.unstack()
for fraction in range(1,10):
    bam_replacement = ".0{}.bam".format(fraction)
    ip_file_list[('unshuffled_bam', fraction / 10.)] = ip_file_list['unshuffled_bam', 1.0].apply(lambda x: downsample_path + os.path.basename(x).replace(".bam", bam_replacement))
    
ip_file_list = ip_file_list.stack()

In [20]:
#run only files ready to be run
origional_files = ip_file_list.xs(1.0, level="fraction")
ungenerated_files = origional_files[~origional_files.unshuffled_bam.apply(os.path.exists)]
print len(ungenerated_files)

0


In [21]:
unrun_ip_file_list = ip_file_list[~(ip_file_list.unshuffled_bam.apply(os.path.exists))]
unrun_items = unrun_ip_file_list.groupby(level=["ENCODE_ID", "RBP", "rep"]).groups.keys()

In [22]:
#Can't sort because then rumdup doesn't work anymore...

def format_unshuffled_downsample(key):
    unshuffled_bams = ip_file_list.loc[key].unshuffled_bam
    #Makes a big assumption about the order of everything, this will probably come back to bite me later
    rep = "downsample_bam.py  --no_sort '--bam' '{}'  '--bam01' '{}' '--bam02' '{}'  '--bam03' '{}'  '--bam04' '{}' '--bam05' '{}'  '--bam06' '{}'  '--bam07' '{}'  '--bam08' '{}'  '--bam09' '{}'".format(*unshuffled_bams.values)
    return rep

downsampled_list = []
for key in unrun_items:
    downsampled_list.append(format_unshuffled_downsample(key))

In [23]:
#These files don't exist because they haven't been processed yet
len(downsampled_list)

0

In [24]:
num_slices = 500
full_process_list = []
for x in range(num_slices):
    cur_list = downsampled_list[x::num_slices]
    if len(cur_list) > 0:
        full_process_list.append(" && ".join(cur_list))

In [25]:
len(full_process_list)

0

In [26]:
job = qtools.Submitter(commands=full_process_list, 
                 job_name="downsample_unshuffled", 
                 sh_filename="/home/gpratt/projects/idr/scripts/downsample_unshuffled.sh",
                array=True,
                walltime="6:00:00",
                out_filename="/home/gpratt/projects/idr/scripts/downsample_unshuffled.out",
                err_filename="/home/gpratt/projects/idr/scripts/downsample_unshuffled.err",
                queue="home-yeo")
job.job()

running 0 tasks as an array-job.


0

# Remove PCR Duplciates
(Don't run until previous code is done running)

In [ ]:
null_ip_file_list = ip_file_list.rmdup_bam.isnull()
ip_file_list.loc[null_ip_file_list, 'rmdup_bam'] = ip_file_list.loc[null_ip_file_list].unshuffled_bam.apply(format_rmrep_out_name)
ip_file_list['sorted_bam'] = ip_file_list.unshuffled_bam.apply(sorted_file_name)

In [ ]:
remaining_jobs = []

unsorted_files = ip_file_list[~ip_file_list.sorted_bam.apply(os.path.exists)]
remaining_jobs += list(unsorted_files.unshuffled_bam.apply(sort_and_index).values)

#This is a bit silly, but I need to seperate out the fully run file from the unrun ones
import functools
format_full_rmrep = functools.partial(format_rmrep, shuffle=True)

unrmduped_files = ip_file_list[~ip_file_list["rmdup_bam"].apply(os.path.exists)]
remaining_jobs += list(unrmduped_files.unshuffled_bam.xs(1.0, level="fraction").apply(format_full_rmrep).values)
remaining_jobs += list(unrmduped_files[~unrmduped_files.index.get_level_values(level="fraction").isin([1.0])].unshuffled_bam.apply(format_rmrep).values)

In [ ]:
len(remaining_jobs)

In [ ]:
num_slices = 500
full_process_list = []
for x in range(num_slices):
    cur_list = remaining_jobs[x::num_slices]
    if len(cur_list) > 0:
        full_process_list.append(" && ".join(cur_list))

In [ ]:
len(full_process_list)

In [ ]:
job = qtools.Submitter(commands=full_process_list, 
                 job_name="downsample_rmdup", 
                 sh_filename="/home/gpratt/projects/idr/scripts/downsample_rmdup.sh",
                array=True,
                walltime="8:00:00",
                out_filename="/home/gpratt/projects/idr/scripts/downsample_rmdup.out",
                err_filename="/home/gpratt/projects/idr/scripts/downsample_rmdup.err",
                       queue="home-yeo")
job.job()

# Try the adjectency graph approach

In [ ]:
umi_out_dir = "/home/gpratt/projects/idr/analysis/umi_downsampling"

def umi_name(fn):
    out_file_base = os.path.join(umi_out_dir, os.path.splitext(os.path.basename(fn))[0])
    out_file = out_file_base + ".dedup.bam"
    return out_file

def sorted_umi_name(fn):
    out_file = umi_name(fn)
    sorted_file = os.path.splitext(out_file)[0] + ".sorted.bam"
    return sorted_file

def umi_downsample(fn):
    out_file = umi_name(fn)
    sorted_file = sorted_umi_name(fn)
    
    out_file_base = os.path.join(umi_out_dir, os.path.splitext(os.path.basename(fn))[0])
    
    refrormatted_fn = out_file_base + ".adjusted.bam"
    metrics_name = out_file_base + ".dedup"
    reformat_command = "python /home/gpratt/gscripts/gscripts/clipseq/adjust_umi_location.py --in_bam {} --out_bam {}".format(fn, refrormatted_fn)
    index_command = "samtools index {}".format(refrormatted_fn)
    sort_command = "samtools sort {} -o {} 2> /dev/null".format(out_file, sorted_file)
    index_command_2 = "samtools index {}".format(sorted_file)
    #umi_command = "umi_tools dedup -I {} --output-stats {} -S {}  --paired".format(refrormatted_fn, metrics_name, out_file)
    #Apparently the stats output slows things down, getting rid of it for now
    umi_command = "umi_tools dedup -I {}  -S {} --paired".format(refrormatted_fn, out_file)

    return " && ".join([reformat_command, index_command, umi_command, sort_command, index_command_2])
    #return " && ".join([sort_command, index_command_2])

In [ ]:
ip_file_list['directional_bam'] = ip_file_list.sorted_bam.apply(sorted_umi_name)

In [ ]:
unrun_files = ip_file_list[~ip_file_list['directional_bam'].apply(os.path.exists)]
remaining_jobs = list(unrun_files.sorted_bam.apply(umi_downsample).values)

In [ ]:
len(remaining_jobs)

In a perfect world I'd re-run all of this downsampling stuff with my fixed bam splitter, some 01 datasets don't have any reads

In [ ]:
num_slices = 500
full_process_list = []
for x in range(num_slices):
    cur_list = remaining_jobs[x::num_slices]
    if len(cur_list) > 0:
        full_process_list.append(" && ".join(cur_list))

In [ ]:
len(full_process_list)

In [ ]:
out_root = "adjecency_matrix_rmdup"
job = qtools.Submitter(commands=full_process_list, 
                 job_name=out_root, 
                 sh_filename="/home/gpratt/projects/idr/scripts/{}.sh".format(out_root),
                array=True,
                walltime="48:00:00",
                out_filename="/home/gpratt/projects/idr/scripts/{}.out".format(out_root),
                err_filename="/home/gpratt/projects/idr/scripts/{}.err".format(out_root),
                       queue="home-yeo")
job.job()

print "/home/gpratt/projects/idr/scripts/{}.sh".format(out_root)

# Count and process all generated data

I'm not getting all data because the umi stuff is crashing on datasets with very low read numbers,  I'm calling this fine because we don't want that data anyway. 

In [ ]:
run_ip_file_list = ip_file_list[['sorted_bam', 'directional_bam', 'rmdup_bam']]

In [ ]:
run_ip_file_list.head().applymap(os.path.exists)

In [ ]:
files_cant_process = run_ip_file_list.applymap(os.path.exists)
files_cant_process[~files_cant_process.apply(all, axis=1)]

In [ ]:
genomic_chroms = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr8', 'chr9',
'chr10','chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr18', 'chr19'
'chr20','chr21', 'chr22', 'chr23', 'chrX', 'chrY', 'chrM']

def get_chrom_mapped_reads(fn):
    samtool = pysam.Samfile(fn)

    count = 0
    for read in samtool:
        if read.reference_name in genomic_chroms:
            count += 1
    return count

In [ ]:
#If I need to re-run this I'll want to switch to an idxstats approach, my current appraoch took like a week to re-process

In [ ]:
%%timeit
get_chrom_mapped_reads(run_ip_file_list['sorted_bam'][0])

In [ ]:
%%timeit
pysam.idxstats("/home/gpratt/projects/idr/analysis/downsample_unshuffled_v2/Sample-202_S5_R1.C01_202_01_PTBP1.adapterTrim.round2.rmRep.sorted.bam")

In [ ]:
counts_list = counts_list.drop("directional_bam", axis=1)

In [ ]:
counts_list = run_ip_file_list.copy()
counts_list['sorted_bam'] = run_ip_file_list['sorted_bam'].apply(encode_helpers.get_mapped_reads)
counts_list['sorted_bam_genome_unique'] = run_ip_file_list['sorted_bam'].apply(get_chrom_mapped_reads)

counts_list['directional_bam'] = run_ip_file_list['directional_bam'].apply(encode_helpers.get_mapped_reads)
counts_list['rmdup_bam'] = run_ip_file_list['rmdup_bam'].apply(encode_helpers.get_mapped_reads)
counts_list['rmdup_bam_genome_unique'] = run_ip_file_list['rmdup_bam'].apply(get_chrom_mapped_reads)

#Correct for the fact that I'm counting mapped reads, not mapped pairs here
counts_list_adjusted = counts_list / 2

counts_list_adjusted.to_csv("downsample_counts_v5.txt")
run_ip_file_list.to_csv("run_file_list_v5.txt")

There are some issues with duplicate IDs, below is a gchat from Eric detailing what version to use in this study (can ignore now that we have the master list)

1. 235 submitted 4000
1. 284 - correct 4000 run is labeled “284_01_4000_fixed_DDX21” cid 284x4000fix
1. 285 submitted 4000
1. 323 not submitted, use 4000
1. 333 not submitted, use 4000
1. 355 not submitted, use 4000
1. 363 not submitted, use 4000
1. 374 4000 is only rep1, use 2500
1. 386 not submitted, use 4000
1. 390 submitted 4000
1. 403 not submitted, use 4000
1. 451 not submitted, use 4000
1. 485 not submitted, use 4000
1. 486 not submitted, use 4000
1. 632 - submitted 632x, labeled 632_01_bc2rev_SUB1


In [ ]:
new_index = []
for ENCODE_ID, RBP, rep, fraction in counts_list_adjusted.index:
    split_id = ENCODE_ID.split("_")
    uID = split_id[0]
    bio_rep = "_".join(split_id[1:])
    
    new_index.append([uID, RBP, bio_rep, rep, fraction])
    
counts_list_final = counts_list_adjusted.copy()
counts_list_final.index = pd.MultiIndex.from_tuples(new_index, 
                                                    names=["RBP_ID", "RBP", "bio_rep", "tech_rep", "fraction"])

In [ ]:
counts_list_final = counts_list_final.reset_index()
counts_list_final = counts_list_final.set_index(['RBP_ID', "RBP", "bio_rep", "tech_rep", "fraction"])

In [ ]:
#Uncollapsed: raw data
#directional umi_tools dedup directional method
#unique my unique collapsing method
#counts_list_final.columns = ['uncollapsed', 'directional', 'unique']
counts_list_final.columns = ['uncollapsed', 'unique', 'genomic_uncollapsed', 'genomic_unique']

counts_list_final.to_csv("downsample_counts_full_v5.csv")